## 3.3.1 ⽣成数据集

In [2]:
from mxnet import autograd, nd

num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = nd.random.normal(scale=1, shape=(num_examples, num_inputs))
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += nd.random.normal(scale=0.01, shape=labels.shape)

## 3.3.2 读取数据集

In [5]:
from mxnet.gluon import data as gdata

batch_size = 10
dataset = gdata.ArrayDataset(features, labels)
data_iter = gdata.DataLoader(dataset, batch_size, shuffle=True)

In [6]:
for X, y in data_iter:
    print(X, y)
    break


[[ 0.54932624  0.6688221 ]
 [ 1.9469851   0.04463983]
 [-0.15107684  0.95578885]
 [ 0.3893424  -0.07695507]
 [ 0.21828133 -0.39872193]
 [-0.17266597 -1.1916859 ]
 [-0.8288414   0.9591693 ]
 [-0.46734235  0.11105727]
 [-0.0842585  -0.73394597]
 [-0.52118605  0.23914368]]
<NDArray 10x2 @cpu(0)> 
[ 3.0219407  7.9360065  0.6560518  5.2391057  5.9982696  7.917271
 -0.7172271  2.8801243  6.529941   2.3400323]
<NDArray 10 @cpu(0)>


## 3.3.3 定义模型

In [7]:
from mxnet.gluon import nn

net = nn.Sequential()  # 串联各个层的容器

In [8]:
net.add(nn.Dense(1))  # 全连接层输出为1

## 3.3.4 初始化模型参数

In [10]:
from mxnet import init

net.initialize(init.Normal(sigma=0.01))  # 0均值 0.01方差

## 3.3.5 定义损失函数

In [12]:
from mxnet.gluon import loss as gloss

loss = gloss.L2Loss()

## 3.3.6 定义优化算法

In [13]:
from mxnet import gluon

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

## 3.3.7 训练模型

In [14]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print('epoch %d, loss: %f' % (epoch, l.mean().asnumpy()))

epoch 1, loss: 0.034962
epoch 2, loss: 0.000127
epoch 3, loss: 0.000049


In [15]:
dense = net[0]
true_w, dense.weight.data()

([2, -3.4],
 
 [[ 1.9997265 -3.3994873]]
 <NDArray 1x2 @cpu(0)>)

In [16]:
true_b, dense.bias.data()

(4.2,
 
 [4.2000127]
 <NDArray 1 @cpu(0)>)

In [17]:
dense.weight.grad()


[[ 0.00541917 -0.05896775]]
<NDArray 1x2 @cpu(0)>